<a href="https://colab.research.google.com/github/pnkjjha/BuildNExperiment/blob/main/Structured_Streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=acdb90badda32647122a3796c756237cc5c88a80a3ba5d6ca17a6a1bc06f117d
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pandas import json_normalize
import pyspark.sql.functions as F
import pyspark.sql.types as T
# import random
import requests
import json
import os

In [3]:
spark=SparkSession.builder.appName('test').getOrCreate()

In [ ]:
# url = "https://api.carbonintensity.org.uk/intensity"

# while switch:
#   response = requests.get(url)
#   json_normalize(response.json())
#   message = response.json()
#   json_normalize(message['data']).rename(columns={"intensity.forecast": "intensity_forecast",\
#                                                   "intensity.actual": "intensity_actual",\
#                                                   "intensity.percentile": "intensity_percentile",\
#                                                   "intensity.index": "intensity_index"\
#                                                   }).to_csv("intensity_"+str(random.randint(1,10000))+".csv", index=False)


In [4]:
s = requests.Session()

i = 0
url = "https://stream.wikimedia.org/v2/stream/recentchange"
with s.get(url, stream=True) as resp:
  for line in resp.iter_lines(1024,decode_unicode=True):
    # Check if line has some bytes in it
    if line:
      # convert line into json
      # get data lines only
      if line[:4] == "data":
          # print(json.loads(line[5:]))
          with open("recentChanges.json","a",encoding="utf-8") as f:
              f.writelines(line[5:]+ os.linesep)
          i = i + 1
    if i>=10 :
      f.close()
      break

In [18]:
cSchema = T.StructType([
     T.StructField("$schema", T.StringType())
    ,T.StructField("meta", T.StructType(
        [T.StructField("uri", T.StringType())
        ,T.StructField("request_id", T.StringType())
        ,T.StructField("id", T.StringType())
        ,T.StructField("dt", T.DateType())
        ,T.StructField("domain", T.StringType())
        ,T.StructField("stream", T.StringType())
        ,T.StructField("topic", T.StringType())
        ,T.StructField("partition", T.IntegerType())
        ,T.StructField("offset", T.IntegerType())
        ]
    ))
    ,T.StructField("id", T.IntegerType())
    ,T.StructField("type", T.DateType())
    ,T.StructField("namespace", T.StringType())
    ,T.StructField("title", T.StringType())
    ,T.StructField("title_url", T.StringType())
    ,T.StructField("comment", T.StringType())
    ,T.StructField("timestamp", T.IntegerType())
    ,T.StructField("user", T.StringType())
    ,T.StructField("bot", T.BooleanType())
    ,T.StructField("notify_url", T.StringType())
    ,T.StructField("minor", T.StringType())
    ,T.StructField("patrolled", T.StringType())
    ,T.StructField("length",T.StructType([
         T.StructField("old",T.IntegerType()
        ,T.StructField("new",T.IntegerType())
        )]))
    ,T.StructField("revision", T.StructType([
         T.StructField("old",T.IntegerType())
        ,T.StructField("new",T.IntegerType())
    ]))
    ,T.StructField("server_url", T.StringType())
    ,T.StructField("server_name", T.StringType())
    ,T.StructField("server_script_path", T.StringType())
    ,T.StructField("wiki", T.StringType())
    ,T.StructField("parsedcomment", T.StringType())
])

In [17]:
iSchema = spark.read.load(".", format="json").schema

In [28]:
#create Dataframe representing the stream of input lines from connection to localhost:9999
intensity = spark.readStream.format('json').schema(iSchema).options(path=".").option('spark.sql.legacy.pathOptionBehavior.enabled',True).option("maxFilesPerTrigger", 1).load("recentChanges.json")
intensity

AnalysisException: There is a 'path' option set and load() is called with a path parameter. Either remove the path option, or call load() without the parameter. To ignore this check, set 'spark.sql.legacy.pathOptionBehavior.enabled' to 'true'.

In [25]:
intensity.groupby("type").count().writeStream.format("console").outputMode("complete").start().awaitTermination()


StreamingQueryException: [STREAM_FAILED] Query [id = a123c13d-aa32-44ff-a521-ab28cbd9a15e, runId = 6587c5af-f2e5-4a5f-8bf2-0a526f9644f7] terminated with exception: Option 'basePath' must be a directory